In [1]:
from hash import *
import numpy as np
from scipy.stats import rankdata

In [2]:
def new_hash_adjacency_matrix_helper(matrix):
    r_collisions = row_collisions(matrix)
    r_diag = np.diag(r_collisions).reshape(1, -1).repeat(r_collisions.shape[0], axis=0)

    return new_sort_matrices(r_collisions, r_diag, matrix)

In [3]:
def old_hash_adjacency_matrix_helper(matrix):
    r_collisions = row_collisions(matrix)
    r_diag = np.diag(r_collisions).reshape(1, -1).repeat(r_collisions.shape[0], axis=0)

    return old_sort_matrices(r_collisions, r_diag, matrix)

In [4]:
def old_sort_matrices(r_collisions, r_diag, matrix):
    stacked_matrices = np.stack([r_collisions, r_diag], axis=2)

    #print(f"row collisions: {r_collisions}")
    #print(f"diagonals: {r_diag}")
    #print(f"stacked 1: {stacked_matrices}")

    for row_index in range(stacked_matrices.shape[0]):
        stacked_matrices[row_index] = np.sort(stacked_matrices[row_index], axis=0)

    #print(f"stacked 2: {stacked_matrices}")

    matrix = np.array([x for _, _, x in sorted(
        zip(stacked_matrices[:, :, 0].tolist(), stacked_matrices[:, :, 1].tolist(), matrix.tolist()))])

    return matrix

In [5]:
def new_sort_matrices(r_collisions, r_diag, matrix):
    #stacked_matrices = np.stack([r_collisions, r_diag], axis=2)
    stacked_matrices = np.stack([np.sort(r_collisions), r_diag], axis=2) #
    #stacked_matrices = np.sort(stacked_matrices, axis=1)

    #stacked_matrices = np.stack([np.diag(r_collisions).argsort(), r_collisions[:, 0].argsort()],axis=1)

    matrix = np.array([x for _, _, x in sorted(
        zip(stacked_matrices[:, :, 0].tolist(), stacked_matrices[:, :, 1].tolist(), matrix.tolist()))])

    #matrix = matrix[stacked_matrices[:, 0].argsort()]

    return matrix

In [6]:
def rank_repeat_based_generic(arr):
    sidx = np.argsort(arr,kind='mergesort')
    idx = np.concatenate(([0],np.flatnonzero(np.diff(arr[sidx]))+1,[arr.size]))
    return np.repeat(idx[:-1],np.diff(idx))[sidx.argsort()]

In [7]:
def rankmin(x):
    u, inv, counts = np.unique(x, return_inverse=True, return_counts=True)
    csum = np.zeros_like(counts)
    csum[1:] = counts[:-1].cumsum()
    return csum[inv]

In [8]:
sizes=4
matrix = np.random.randint(2, size=(sizes, sizes))
matrix

array([[0, 1, 0, 1],
       [1, 0, 0, 0],
       [0, 1, 1, 0],
       [1, 0, 1, 0]])

In [9]:
m1 = old_hash_adjacency_matrix_helper(matrix)
m2 = new_hash_adjacency_matrix_helper(matrix)
np.allclose(m1, m2)

True

In [336]:
r_collisions = row_collisions(matrix)
r_diag = np.diag(r_collisions).reshape(1, -1).repeat(r_collisions.shape[0], axis=0)

In [337]:
r_collisions

array([[3, 2, 2, 0],
       [2, 2, 1, 0],
       [2, 1, 3, 1],
       [0, 0, 1, 1]])

In [342]:
np.sort(r_collisions)

array([[0, 2, 2, 3],
       [0, 1, 2, 2],
       [1, 1, 2, 3],
       [0, 0, 1, 1]])

In [338]:
r_diag

array([[3, 2, 3, 1],
       [3, 2, 3, 1],
       [3, 2, 3, 1],
       [3, 2, 3, 1]])

In [ ]:
stacked_matrices = np.stack([np.sort(r_collisions), r_diag], axis=2)

In [285]:
m1

array([[0, 1, 1, 1],
       [1, 1, 1, 0],
       [1, 0, 1, 1],
       [1, 0, 1, 1]])

In [286]:
m2

array([[1, 1, 1, 0],
       [1, 0, 1, 1],
       [0, 1, 1, 1],
       [1, 0, 1, 1]])

In [301]:
temp_r = np.random.randint(4, size=(5))
temp_r

array([2, 2, 3, 1, 3])

In [318]:
%timeit rank_repeat_based_generic(temp_r)

36.1 µs ± 2.37 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [319]:
%timeit rankdata(temp_r, method='min')

68.6 µs ± 1.38 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [320]:
%timeit rankmin(temp_r)

51.8 µs ± 821 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [82]:
sizes=4
matrix = np.random.randint(2, size=(sizes, sizes))
matrix

array([[1, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [1, 1, 0, 1]])

In [83]:
row_collisions(matrix)

array([[2, 1, 0, 2],
       [1, 1, 0, 1],
       [0, 0, 1, 0],
       [2, 1, 0, 3]])

In [111]:
np.argsort(row_collisions(matrix),axis=1)

array([[2, 1, 0, 3],
       [2, 0, 1, 3],
       [0, 1, 3, 2],
       [2, 1, 0, 3]], dtype=int64)

In [110]:
np.sort(row_collisions(matrix),axis=1)

array([[0, 1, 2, 2],
       [0, 1, 1, 1],
       [0, 0, 0, 1],
       [0, 1, 2, 3]])

In [107]:
#np.sort(row_collisions(matrix)).argsort(axis=0)
np.sort(row_collisions(matrix))[0, :].argsort()

array([0, 1, 2, 3], dtype=int64)

In [86]:
row_swaps = row_collisions(matrix)[:, 0].argsort()
row_swaps

array([2, 1, 0, 3], dtype=int64)

In [84]:
row_swaps = row_collisions(matrix)[0, :].argsort()
row_swaps

array([2, 1, 0, 3], dtype=int64)

In [60]:
col_swaps = row_collisions(matrix.T)[0, :].argsort()
col_swaps

array([1, 2, 0, 3], dtype=int64)

In [77]:
row_collisions(matrix[row_swaps])

array([[2, 0, 1, 1],
       [0, 1, 1, 1],
       [1, 1, 3, 2],
       [1, 1, 2, 3]])

In [76]:
row_collisions(matrix[:, col_swaps].T)

array([[2, 1, 1, 1],
       [1, 2, 1, 1],
       [1, 1, 2, 2],
       [1, 1, 2, 3]])

In [35]:
matrix[:, col_swaps]

array([[1, 0, 0, 1],
       [1, 1, 0, 1],
       [0, 0, 1, 1],
       [1, 1, 1, 1]])

In [73]:
#swapped_matrix = matrix[row_swaps[:, np.newaxis], col_swaps]
swapped_matrix = matrix[row_swaps][:, col_swaps]
swapped_matrix

array([[1, 1, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 1, 1],
       [0, 1, 1, 1]])

In [74]:
row_collisions(swapped_matrix)

array([[2, 0, 1, 1],
       [0, 1, 1, 1],
       [1, 1, 3, 2],
       [1, 1, 2, 3]])

In [75]:
row_collisions(swapped_matrix.T)

array([[2, 1, 1, 1],
       [1, 2, 1, 1],
       [1, 1, 2, 2],
       [1, 1, 2, 3]])

In [40]:
matrix = np.array([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9]])

# Define the indices for row and column swaps
row_swaps = np.array([2, 0, 1])  # New order of rows
col_swaps = np.array([1, 2, 0])  # New order of columns

# Swap both rows and columns using advanced indexing
swapped_matrix = matrix[row_swaps[:, np.newaxis], col_swaps]

print("Original matrix:")
print(matrix)
print("\nSwapped matrix:")
print(swapped_matrix)

Original matrix:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

Swapped matrix:
[[8 9 7]
 [2 3 1]
 [5 6 4]]


In [66]:
matrix[row_swaps][:, col_swaps]

array([[1, 1, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 1, 1],
       [0, 1, 1, 1]])